In [1]:
import sys
sys.path.append("..")

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [7]:
# Import Libraries
from utils import rotate_preserve_size
from loss import angular_loss_mae, sinusoidal_loss
import glob
import os
import numpy as np
import cv2
import random

from tensorflow.keras.models import Model
from tensorflow.keras import layers as L
import tensorflow as tf
import os
import pandas as pd
from tensorflow.keras.applications import Xception, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from loguru import logger
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adadelta
import tensorflow.keras.backend as K

In [3]:
#Define conv base
# conv_base = Xception(weights="imagenet", include_top=False, input_shape=(299, 299, 3))
conv_base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(299, 299, 3))
for layer in conv_base.layers:
    layer.trainable = True

2022-04-15 15:02:32.264056: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-15 15:02:32.264770: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-15 15:02:32.338560: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-04-15 15:02:32.338594: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-04-15 15:02:32.340126: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-04-15 15:02:32.340163: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.

In [4]:
# Define model
img_input = L.Input(shape=(299, 299, 3))
x = conv_base(img_input)
x = L.Flatten()(x)
x = L.Dense(512, activation="relu")(x)
x = L.BatchNormalization()(x)
x = L.Dense(256, activation="relu")(x)
x = L.BatchNormalization()(x)
x = L.Dense(64, activation="relu")(x)
x = L.BatchNormalization()(x)
y = L.Dense(1, activation="linear")(x)
model = Model(img_input, y)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 10, 10, 1280)      4049571   
_________________________________________________________________
flatten (Flatten)            (None, 128000)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               65536512  
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024  

In [5]:
# Batch Generator
class RotGenerator180(Sequence):
    def __init__(self, image_dir, batch_size, dim):
        self.files = glob.glob(os.path.join(image_dir, "*.jpg"))
        self.batch_size = batch_size
        self.dim = dim
        
    def __len__(self):
        if len(self.files) % self.batch_size == 0:
            return len(self.files) // self.batch_size
        return len(self.files) // self.batch_size + 1
    
    def __getitem__(self, idx):
        batch_slice = slice(idx * self.batch_size, (idx + 1) * self.batch_size)
        batch_files = self.files[batch_slice]
        
        X = np.zeros(shape=(len(batch_files), self.dim, self.dim, 3))
        y = np.zeros(shape=(len(batch_files), ))
        
        for i, f in enumerate(batch_files):
            # img = cv2.imread(f)
            angle = float(np.random.choice(range(-180, 180)))
            img = rotate_preserve_size(f, angle, (self.dim, self.dim))
            
            X[i] = img
            y[i] = angle
        
        return X, y
    
    def on_epoch_end(self):
        random.shuffle(self.files)
            

In [6]:
# Batch Generator
class ValidationTestGenerator(Sequence):
    def __init__(self, image_dir, df_label_path, batch_size, dim, mode):
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.dim = dim
        self.mode = mode
        
        df_label = pd.read_csv(df_label_path)
        self.df = df_label[df_label["mode"] == self.mode].reset_index(drop=True)
        
    def __len__(self):
        total = self.df.shape[0]
        if total % self.batch_size == 0:
            return total // self.batch_size
        return total // self.batch_size + 1
    
    def __getitem__(self, idx):
        batch_slice = slice(idx * self.batch_size, (idx + 1) * self.batch_size)
        df_batch = self.df[batch_slice].reset_index(drop=True).copy()
        
        X = np.zeros(shape=(len(df_batch), self.dim, self.dim, 3))
        y = np.zeros(shape=(len(df_batch), ))
        
        for i in range(len(df_batch)):
            angle = df_batch.angle[i]
            path = os.path.join(self.image_dir, df_batch.image[i])
            img = rotate_preserve_size(path, angle, (self.dim, self.dim))
            
            X[i] = img
            y[i] = angle
        
        return X, y
    
    def on_epoch_end(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)
            

In [8]:
def parabolic_loss(y_true, y_pred):
    y_pred = tf.math.floormod(y_pred, 360)
    
    y_pred = K.switch(K.greater(y_pred, 180), y_pred - 360, y_pred)
    y_pred = K.switch(K.less(y_pred, -180), y_pred + 360, y_pred)
    
    return y_pred

In [16]:
a = np.array([[20.0], [30], [40]])
b = np.array([[25.0], [320.0], [-700]])

parabolic_loss(a, b)

<tf.Tensor: shape=(3, 1), dtype=float64, numpy=
array([[ 25.],
       [-40.],
       [ 20.]])>

In [27]:
from utils import rotate_preserve_size
import glob
import os
import numpy as np
import cv2
import random

import tensorflow as tf
import os
import pandas as pd
from tensorflow.keras.utils import Sequence


class RotGenerator(Sequence):
    def __init__(self, image_dir, batch_size, dim):
        self.files = glob.glob(os.path.join(image_dir, "*.jpg"))
        self.batch_size = batch_size
        self.dim = dim
        
    def __len__(self):
        if len(self.files) % self.batch_size == 0:
            return len(self.files) // self.batch_size
        return len(self.files) // self.batch_size + 1
    
    def __getitem__(self, idx):
        batch_slice = slice(idx * self.batch_size, (idx + 1) * self.batch_size)
        batch_files = self.files[batch_slice]
        
        # X = np.zeros(shape=(len(batch_files), self.dim, self.dim, 3))
        # y = np.zeros(shape=(len(batch_files), ))

        X = []
        y = []
        
        for i, f in enumerate(batch_files):
            try:
                angle = float(np.random.choice(range(0, 360)))
                img = rotate_preserve_size(f, angle, (self.dim, self.dim))
                img = np.expand_dims(img, axis=0)
                X.append(img)
                # X[i] = img
                # y[i] = angle
                y.append(angle)

            except:
                pass
        
        X = np.concatenate(X, axis=0)
        y = np.array(y)

        return X, y
    
    def on_epoch_end(self):
        random.shuffle(self.files)
            

class ValidationTestGenerator(Sequence):
    def __init__(self, image_dir, df_label_path, batch_size, dim, mode):
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.dim = dim
        self.mode = mode
        
        df_label = pd.read_csv(df_label_path)
        self.df = df_label[df_label["mode"] == self.mode].reset_index(drop=True)
        
    def __len__(self):
        total = self.df.shape[0]
        if total % self.batch_size == 0:
            return total // self.batch_size
        return total // self.batch_size + 1
    
    def __getitem__(self, idx):
        batch_slice = slice(idx * self.batch_size, (idx + 1) * self.batch_size)
        df_batch = self.df[batch_slice].reset_index(drop=True).copy()
        
        # X = np.zeros(shape=(len(df_batch), self.dim, self.dim, 3))
        # y = np.zeros(shape=(len(df_batch), ))

        X = []
        y = []
        
        for i in range(len(df_batch)):
            try:
                angle = df_batch.angle[i]
                path = os.path.join(self.image_dir, df_batch.image[i])
                img = rotate_preserve_size(path, angle, (self.dim, self.dim))
                img = np.expand_dims(img, axis=0)
                
                # X[i] = img
                # y[i] = angle

                X.append(img)
                y.append(angle)

            except:
                pass
        
        X = np.concatenate(X, axis=0)
        y = np.array(y)

        return X, y
    
    def on_epoch_end(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)

In [28]:
train_gen = RotGenerator("/data/chandanp/train2017/", 2, 299)

In [29]:
train_gen.__getitem__(2)[0].shape

(2, 299, 299, 3)

In [8]:
# train
model.compile(loss=sinusoidal_loss, optimizer=Adadelta(learning_rate=0.1), metrics=[angular_loss_mae])

train_gen = RotGenerator("/data/chandanp/train2017/", 32, 299)
val_gen = ValidationTestGenerator(image_dir="/data/subhadip/data/validation-test/", 
                                  df_label_path="/data/subhadip/data/validation-test.csv",
                                  batch_size=128, dim=299, mode="valid")
cp = ModelCheckpoint("/data/subhadip/weights/model-en-ang-loss.h5", save_weights_only=False, 
                     save_best_only=True, monitor="loss")
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5)
es = EarlyStopping(monitor="val_loss", patience=5)
model.fit(train_gen, validation_data=val_gen, epochs=10000, callbacks=[cp, es, reduce_lr])

2022-03-28 18:27:41.387384: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-28 18:27:41.405119: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3597875000 Hz


Epoch 1/10000
  15/3697 [..............................] - ETA: 6:26:46 - loss: 19.5522 - angular_loss_mae: 92.6453

KeyboardInterrupt: 